In [12]:
import mediapipe as mp
import cv2
import numpy as np

In [13]:
from turtle import right
global_boundingbox_coord = None
global_distance = None
dynamic_bounding_box = True

def cropping(holistic, mp_holistic, image):
    mp_drawing = mp.solutions.drawing_utils
    drawing = {"mp_drawing": mp_drawing}
    result = holistic.process(image)
    # if result.pose_landmarks == None:
    #     return []

    image = body_tracking(result.pose_landmarks, image)

    return image

def drawing(holistic, mp_holistic, image):
    mp_drawing = mp.solutions.drawing_utils
    drawing = {"mp_drawing": mp_drawing}
    result = holistic.process(image)
    drawing["image"] = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    face_drawing = drawing.copy()
    pose_drawing = drawing.copy()
    left_hand_drawing = drawing.copy()
    right_hand_drawing = drawing.copy()
    face_drawing["landmarks"] = result.face_landmarks
    face_drawing["connections"] = mp_holistic.FACEMESH_TESSELATION
    left_hand_drawing["landmarks"] = result.left_hand_landmarks
    right_hand_drawing["landmarks"] = result.right_hand_landmarks
    left_hand_drawing["connections"] = mp_holistic.HAND_CONNECTIONS
    right_hand_drawing["connections"] = left_hand_drawing["connections"]
    pose_drawing["landmarks"] = result.pose_landmarks
    pose_drawing["connections"] = mp_holistic.POSE_CONNECTIONS
    im = draw_landmarks(image, face_drawing)
    im = draw_landmarks(im, left_hand_drawing)
    im = draw_landmarks(im, right_hand_drawing)
    im = draw_landmarks(image, pose_drawing)

    # print(im.shape)
    return im

def extract_data(holistic, mp_holistic, image):
    result = holistic.process(image)
    face_landmarks = result.face_landmarks
    left_hand_landmarks = result.left_hand_landmarks
    right_hand_landmarks = result.right_hand_landmarks
    pose_landmarks = result.pose_landmarks
    mouth_coordinates = face_landmarks_data(face_landmarks)
    left_hand_coordinates = hand_landmarks_data(left_hand_landmarks)
    right_hand_coordinates = hand_landmarks_data(right_hand_landmarks)
    pose_coordinates = pose_landmarks_data(pose_landmarks)
    coordinates_collection = mouth_coordinates + left_hand_coordinates + right_hand_coordinates + pose_coordinates

    return coordinates_collection

def face_landmarks_data(landmarks):
    coordinates = []
    if landmarks:
        for i in mouth_indices():
            landmark = landmarks.landmark[i]
            coordinates.append(landmark.x)
            coordinates.append(landmark.y)
            coordinates.append(landmark.z)
    else:
        for i in range(0, len(mouth_indices())):
            for i in range(0, 3):
                coordinates.append(0)

    return coordinates

def hand_landmarks_data(landmarks):
    coordinates = []
    if landmarks:
        for i in landmarks.landmark:
            coordinates.append(i.x)
            coordinates.append(i.y)
            coordinates.append(i.z)
    else:
        for i in range(0, 21):
            for i in range(0, 3):
                coordinates.append(0)
    
    return coordinates

def pose_landmarks_data(landmarks):
    coordinates = []
    if landmarks:
        for i in landmarks.landmark:
            coordinates.append(i.x)
            coordinates.append(i.y)
            coordinates.append(i.z)
    else:
        for i in range(0, 33):
            for i in range(0, 3):
                coordinates.append(0)
    
    return coordinates

def body_tracking(pose, image):
    centroid_indices = [0, 11, 12, 23, 24]
    im_h = image.shape[0]
    im_w = image.shape[1]
    if pose is None and global_boundingbox_coord is None:
        return image
    elif pose is None and global_boundingbox_coord is not None:
        x, y, z = global_boundingbox_coord
    else:
        x, y, z = find_body_centroid(pose, centroid_indices)
    if dynamic_bounding_box == True:
        distance = find_distance(pose, im_w, im_h)
        center_x = int(x*im_w)
        center_y = int(y*im_h)
        w, h = int(distance), int(distance)
    else:
        w, h = int(500), int(500)
    box_coord = [center_y-h, center_y+h, center_x-w, center_x+w]
    top, bottom, left, right = padding(box_coord, im_h, im_w)
    image = cv2.copyMakeBorder(
        image, top, bottom, left, right, cv2.BORDER_CONSTANT, None, value=0)
    img_cpy = image.copy()
    return img_cpy[center_y - h + top:center_y + h + bottom + top, center_x-w + left:center_x + w + right + left]

def mouth_indices():
    return [0,13,14,17,37,39,40,61,78,80,81,82,84,87,88,91,95,146,178,181,185,191,267,269,270,291,308,310,311,312,314,317,318,321,324,375,402,405,409,415]

def padding(box_coord, im_h, im_w):
    paddings = np.zeros((4)).astype(dtype=np.int32)
    if box_coord[0] < 0:
        paddings[0] = abs(box_coord[0])
    if box_coord[1] > im_h:
        paddings[1] = abs(box_coord[1] - im_h)
    if box_coord[2] < 0:
        paddings[2] = abs(box_coord[2])
    if box_coord[3] > im_w:
        paddings[3] = abs(box_coord[3] - im_w)

    return paddings

def find_body_centroid(landmarks, indices):
    main_body = indices
    if landmarks:
        x_bodies = []
        y_bodies = []
        z_bodies = []
        for i in main_body:
            x_bodies.append(landmarks.landmark[i].x)
            y_bodies.append(landmarks.landmark[i].y)
            z_bodies.append(landmarks.landmark[i].z)
        global_boundingbox_coord = [x_bodies, y_bodies, z_bodies]
        return np.average(x_bodies), np.average(y_bodies), np.average(z_bodies)

def find_distance(landmarks, im_w, im_h):
    indices_a = [11, 12]
    indices_b = [23, 24]
    centroid_a = np.array(find_body_centroid(landmarks, indices_a))
    centroid_b = np.array(find_body_centroid(landmarks, indices_b))
    point_a = pixel_coordinate_convertion(centroid_a, im_w, im_h)
    point_b = pixel_coordinate_convertion(centroid_b, im_w, im_h)
    sum_sq = np.sum(np.square(point_a - point_b))
    euclidean = np.sqrt(sum_sq)

    return euclidean

def pixel_coordinate_convertion(coordinates, w, h):
    x = int(coordinates[0] * w)
    y = int(coordinates[1] * h)
    coordinates[0] = x
    coordinates[1] = y

    return coordinates

def draw_landmarks(image, params):
    mp_drawing = params["mp_drawing"]
    landmarks = params["landmarks"]
    connections = params["connections"]
    landmarks_drawing_spec = mp_drawing.DrawingSpec(
        color=[255, 0, 0],
        thickness=2,
        circle_radius=2,
    )
    connection_drawing_spec = mp_drawing.DrawingSpec(
        color=[0, 255, 00],
        thickness=1,
        circle_radius=2,
    )
    mp_drawing.draw_landmarks(
        image=image,
        landmark_list=landmarks,
        connections=connections,
        landmark_drawing_spec=landmarks_drawing_spec,
        connection_drawing_spec=connection_drawing_spec
    )

    return image

def capture(path):
    cap = cv2.VideoCapture(path)
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic()
    cropped = []
    with holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
                im_h = frame.shape[0]
                im_w = frame.shape[1]
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image = cropping(holistic, mp_holistic, image)
                if len(image) == 0:
                    return []
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                cropped.append(image)
                key = cv2.waitKey(1)
                if key == ord("q"):
                    break
            else:
                break

    mp_holistic2 = mp.solutions.holistic
    holistic2 = mp_holistic2.Holistic()
    frame_index = 0
    frame_datas = []
    with holistic2:
        for i in cropped:
            image = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
            coordinates_data = extract_data(holistic2, mp_holistic2, image)
            frame_data = {
                "frame_index": frame_index,
                "skeleton": [
                    {
                        "pose": coordinates_data
                    }
                ]
            }

            frame_datas.append(frame_data)
            key = cv2.waitKey(1)
            if key == ord("q"):
                break
            frame_index += 1

    cap.release()
    cv2.destroyAllWindows()

    return frame_datas

In [14]:
import glob
import os
import json
import time
data_path = "./coordinate_recalculation_xy_negative_reflection"
if not os.path.exists(data_path):
    os.makedirs(data_path)

start = time.time()
label_index = 0
bad_video = []
for i in glob.glob("./raw_video_dataset/*"):
    vid_dir = os.path.join(i, "*.mp4")
    file_index = 0
    print(label_index)
    for j in glob.glob(vid_dir):
        class_name = os.path.basename(os.path.split(j)[0])
        file_name = os.path.basename(os.path.splitext(j)[0])
        save_path = os.path.join(data_path, class_name+"_"+str(file_index)+".json")
        keypoints_data = capture(j)
        if len(keypoints_data) == 0:
            bad = {
                "label_index": file_name,
                "label": class_name,
            }
            bad_video.append(bad)
            continue
        data = {
            "label_index": label_index,
            "label": class_name,
            "data": keypoints_data
        }

        with open(save_path, "w") as fp:
            json.dump(data,fp)
        file_index += 1

    label_index += 1
finish = time.time()
time_elapsed = finish - start

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
